In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Masking
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Dense, LayerNormalization, Dropout, Input, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import legacy as legacy_optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import classification_report
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import os
import optuna
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, classification_report, f1_score


In [ ]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')  # May improve speed on some CPUs


In [ ]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')  # May improve speed on some CPUs


In [ ]:
def read_trc(fpath):
    # read metadata in file header
    df_meta = pd.read_csv(fpath, delimiter='\t', header=0, skiprows=1, nrows=1)
    meta = df_meta.iloc[0].to_dict()
    fps = meta['DataRate']

    # read marker location names
    markers_df = pd.read_csv(fpath, delimiter='\t', header=None, skiprows=3, nrows=1)
    markers = markers_df.iloc[0].dropna().to_numpy()[2:]

    # read marker XYZ locations
    df = pd.read_csv(fpath, delimiter='\t', header=0, skiprows=4)
    df.rename(columns=dict(zip(df.columns[:2], ('n', 't'))), inplace=True)
    df.dropna(how='all', axis=1, inplace=True)

    N = df.shape[0]
    M = len(markers)
    xyz = df.iloc[:,2:].to_numpy().reshape((N, M, 3))
    xyz[:,:,[0,1,2]] = xyz[:,:,[2,1,0]]

    return fps, markers, xyz


def read_mot(fpath):
    with open(fpath, 'r') as f:
        # count = 0
        line = f.readline().strip()
        # while line and line.strip() != 'endheader':
        while line.lower() != 'endheader':
            line = f.readline().strip()
            # count += 1

        # df = pd.read_csv(f, delimiter='\t', header=0, skiprows=count-3)
        df = pd.read_csv(f, delimiter='\t', header=0)

    return df

In [ ]:
def z_score_normalize(data_list):
    """
    Applies z-score normalization to a list of time series arrays.
    Arguments:
        data_list: List of numpy arrays, each of shape (num_timesteps, num_features).
    Returns:
        normalized_data: List of z-score normalized arrays.
        mean: Mean of each feature across all time series.
        std: Standard deviation of each feature across all time series.
    """
    # Concatenate all data to compute global mean and std
    all_data = np.concatenate(data_list, axis=0)
    mean = np.mean(all_data, axis=0)
    std = np.std(all_data, axis=0)
    
    # Normalize each array
    normalized_data = [(data - mean) / (std + 1e-8) for data in data_list]
    return normalized_data, mean, std

def prepare_lstm_input(subject_dict):
    """
    Combines multiple time series for each subject along the time axis, applies z-score normalization,
    and pads them for LSTM input.
    Arguments:
        subject_dict: Dictionary where keys are subject IDs and values are lists of 6 numpy arrays.
                      Each numpy array corresponds to a time series with shape (num_timesteps, num_features).
    Returns:
        padded_input: A numpy array of shape (num_subjects, max_timesteps, num_features).
        subject_ids: List of subject IDs in the order they appear in the padded_input array.
        mean: Mean of features used for z-score normalization.
        std: Standard deviation of features used for z-score normalization.
    """
    # Sort the dictionary by keys to ensure consistent ordering
    subject_ids = sorted(subject_dict.keys())
    
    # Extract and concatenate the 6 time series for each subject along the time axis
    concatenated_series = [
        np.concatenate(subject_dict[subject_id], axis=0) for subject_id in subject_ids
    ]
    
    # Apply z-score normalization to the concatenated series
    normalized_series, mean, std = z_score_normalize(concatenated_series)
    
    # Determine the maximum length for padding
    max_length = max(series.shape[0] for series in normalized_series)
    
    # Pad all normalized time series to the same length
    padded_input = pad_sequences(normalized_series, maxlen=max_length, padding="post", dtype="float32")
    
    return padded_input, subject_ids, mean, std

In [ ]:
def get_label_for_participant_trial(class_df, pid, trial):
    """
    Fetches the label for a given participant and trial based on class_info.csv.
    
    Parameters:
        participant_id (str): The ID of the participant.
        trial (str): The trial name or ID.
    
    Returns:
        int: Classification label (0 or 1).
    """
    label = class_df.loc[class_df['ID'] == pid, 'Class'].values[0]
    return label

In [ ]:
# Directory containing .mot files
root_dir = '/Users/scovitz/datadir/opencap_data'  # path to root data dir 
labels_csv = '/Users/scovitz/datadir/class_info.csv'
class_df = pd.read_csv(labels_csv)

time_series_data = {}  # dictionary of pid, timeseries pairs 

expected_columns = [
        'pelvis_tilt', 'pelvis_list', 'pelvis_rotation', 'pelvis_tx',
        'pelvis_ty', 'pelvis_tz', 'hip_flexion_r', 'hip_adduction_r',
        'hip_rotation_r', 'knee_angle_r', 'ankle_angle_r',
        'subtalar_angle_r', 'mtp_angle_r', 'hip_flexion_l', 'hip_adduction_l',
        'hip_rotation_l', 'knee_angle_l', 'ankle_angle_l',
        'subtalar_angle_l', 'mtp_angle_l', 'lumbar_extension', 'lumbar_bending',
        'lumbar_rotation', 'arm_flex_r', 'arm_add_r', 'arm_rot_r',
        'elbow_flex_r', 'pro_sup_r', 'arm_flex_l', 'arm_add_l', 'arm_rot_l',
        'elbow_flex_l', 'pro_sup_l'
    ]

# Loop through all .mot files
for i in range(129):
        # ensure going in order!
        num = str(i+1).zfill(3)

        for path in Path(root_dir).rglob(f"P{num}*/**/*.mot"): 
            
            # remove upper extremity trials 
            if "brooke" in str(path) or "curls" in str(path) or "arm_rom" in str(path):
                continue 
            
            trial = str(path).split('/')[-3]
            pid = str(path).split('/')[-4]
            activity = str(path).split('/')[-2]
            #print(str(path))
            # Load the .mot file
            
            data = read_mot(str(path))
            data = data[200:]
            #print(data.shape)

            # Ensure the file has the expected columns
            if not all(col in data.columns for col in expected_columns + ['time']):
                raise ValueError(f"File {path} does not contain the expected columns.")
            
            # Drop the 'time' column and keep only relevant features
            data = np.array(data[expected_columns])
            
            if pid not in time_series_data:
                time_series_data[pid + '_' + trial] = []

            time_series_data[(pid + '_' + trial)].append(data) # value = list of timeseries arrays of num_timesteps x num_feats
            print(data.shape, pid, trial, str(path))




In [ ]:
len(time_series_data)
time_series_data

In [ ]:
# Prepare the LSTM input
lstm_input, subject_ids, mean, std = prepare_lstm_input(time_series_data)

# Check the results
print("LSTM Input Shape:", lstm_input.shape)  # (162, max_timesteps, 35)
print("Subject IDs:", subject_ids[:5])  # subject ID ordering, should be in order! 
print("Mean Shape:", mean.shape)  # shape of mean: (35,)
print("Std Shape:", std.shape)  # shape of std: (35,)




In [ ]:
# Convert labels to a numpy array
labels = []
for sid in subject_ids:
    pid = sid.split('_')[0]
    trial = sid.split('_')[1]
    label = get_label_for_participant_trial(class_df, pid, trial)
    labels.append(label)

In [ ]:
# convert to numpy array
labels = np.array(labels)

In [ ]:
# data loader
def data_generator(X, y, batch_size):
    num_samples = len(X)
    while True:
        for offset in range(0, num_samples, batch_size):
            X_batch = X[offset:offset+batch_size]
            y_batch = y[offset:offset+batch_size]
            yield np.array(X_batch), np.array(y_batch)


In [ ]:
# Split data into train, validation, and test sets - 80, 10, 10
def split_data(X, y, test_size=0.1, val_size=0.1):
    """
    Splits the data into training, validation, and testing sets.
    Arguments:
        X: Input features (numpy array).
        y: Labels (numpy array).
        test_size: Proportion of data for the test set.
        val_size: Proportion of data for the validation set from the training set.
    Returns:
        X_train, X_val, X_test, y_train, y_val, y_test3
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_size, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test


# Split data
X_train, X_val, X_test, y_train, y_val, y_test = split_data(lstm_input, labels)


In [ ]:
# QUESTIONS
# - scope of project/priorities? one model? multiple? for 3 ppl?
# - AWS GPU credits? need? way to do local or sherlock? 
# - can we do 80/20 split with data we have train/test? do we need dev? 
# 

In [ ]:
# ADVICE
# spend a lot of time on analysis portion - why did this occur if we were moving forward what would we change 
# try predicting TFT time 

In [ ]:
#### TRYING OPTUNA FOR HYPERPARAM OPTIMIZATION #####

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Input, GlobalAveragePooling1D
# from tensorflow.keras.models import Model
# from tensorflow.keras.regularizers import l2

# class MultiHeadSelfAttention(tf.keras.layers.Layer):
#     def __init__(self, embed_dim, num_heads, regularizer=None):
#         super(MultiHeadSelfAttention, self).__init__()
#         self.embed_dim = embed_dim
#         self.num_heads = num_heads
#         self.projection_dim = embed_dim // num_heads
#         self.query_dense = Dense(embed_dim, kernel_regularizer=regularizer)
#         self.key_dense = Dense(embed_dim, kernel_regularizer=regularizer)
#         self.value_dense = Dense(embed_dim, kernel_regularizer=regularizer)
#         self.combine_heads = Dense(embed_dim, kernel_regularizer=regularizer)

#     def attention(self, query, key, value):
#         score = tf.matmul(query, key, transpose_b=True) / tf.sqrt(tf.cast(self.projection_dim, query.dtype))
#         weights = tf.nn.softmax(score, axis=-1)
#         return tf.matmul(weights, value), weights

#     def split_heads(self, x, batch_size):
#         x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
#         return tf.transpose(x, perm=[0, 2, 1, 3])

#     def call(self, inputs):
#         inputs = tf.cast(inputs, dtype=tf.float32)

#         batch_size = tf.shape(inputs)[0]
#         query = self.query_dense(inputs)
#         key = self.key_dense(inputs)
#         value = self.value_dense(inputs)

#         query = self.split_heads(query, batch_size)
#         key = self.split_heads(key, batch_size)
#         value = self.split_heads(value, batch_size)

#         attention, _ = self.attention(query, key, value)
#         attention = tf.transpose(attention, perm=[0, 2, 1, 3])
#         concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
#         return self.combine_heads(concat_attention)

# class TransformerBlock(tf.keras.layers.Layer):
#     def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, regularizer=None):
#         super(TransformerBlock, self).__init__()
#         self.att = MultiHeadSelfAttention(embed_dim, num_heads, regularizer)
#         self.ffn = tf.keras.Sequential([
#             Dense(ff_dim, activation='relu', kernel_regularizer=regularizer),
#             Dense(embed_dim, kernel_regularizer=regularizer)
#         ])
#         self.layernorm1 = LayerNormalization(epsilon=1e-6)
#         self.layernorm2 = LayerNormalization(epsilon=1e-6)
#         self.dropout1 = Dropout(rate)
#         self.dropout2 = Dropout(rate)

#     def call(self, inputs, training):
#         attn_output = self.att(inputs)
#         attn_output = self.dropout1(attn_output, training=training)
#         out1 = self.layernorm1(inputs + attn_output)
#         ffn_output = self.ffn(out1)
#         ffn_output = self.dropout2(ffn_output, training=training)
#         return self.layernorm2(out1 + ffn_output)


In [ ]:
# import os
# import optuna
# import tensorflow as tf
# from sklearn.utils.class_weight import compute_class_weight
# from sklearn.metrics import f1_score
# import numpy as np
# import matplotlib.pyplot as plt

# # Disable GPU for CPU-only machine
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# # Compute class weights
# class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
# class_weight_dict = dict(enumerate(class_weights))

# # Define the model structure
# def build_model(trial, input_shape):
#     # Suggest hyperparameters
#     embed_dim = trial.suggest_int("embed_dim", 32, 128, step=32)  # Embedding dimension
#     num_heads = trial.suggest_int("num_heads", 2, 8, step=2)      # Number of attention heads
#     ff_dim = trial.suggest_int("ff_dim", 64, 256, step=64)        # Feedforward network dimension
#     num_blocks = trial.suggest_int("num_blocks", 1, 3)            # Number of Transformer blocks
#     dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)  # Dropout rate
#     l2_reg = trial.suggest_float("l2_reg", 1e-4, 1e-2, log=True)  # L2 regularization strength

#     # Input layer
#     inputs = tf.keras.Input(shape=input_shape)
#     x = tf.keras.layers.Dense(embed_dim, kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(inputs)
    
#     # Add Transformer blocks
#     for _ in range(num_blocks):
#         x = TransformerBlock(embed_dim, num_heads, ff_dim, rate=dropout_rate, regularizer=tf.keras.regularizers.l2(l2_reg))(x, training=True)
    
#     # Global pooling and output layer
#     x = tf.keras.layers.GlobalAveragePooling1D()(x)
#     x = tf.keras.layers.Dropout(dropout_rate)(x)
#     outputs = tf.keras.layers.Dense(1, activation="sigmoid", kernel_regularizer=tf.keras.regularizers.l2(l2_reg))(x)

#     # Compile the model
#     learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
#     model = tf.keras.Model(inputs, outputs)
#     model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
#                   loss="binary_crossentropy",
#                   metrics=["accuracy"])
#     return model

# # Define the objective function
# def objective(trial):
#     epochs = trial.suggest_int("epochs", 50, 200, step=50)  # Suggest number of epochs
#     model = build_model(trial, input_shape=(X_train.shape[1], X_train.shape[2]))

#     # Train the model with class weights
#     history = model.fit(
#         X_train, y_train,
#         validation_data=(X_val, y_val),
#         epochs=epochs,
#         batch_size=16,
#         class_weight=class_weight_dict,
#         verbose=0  # Suppress output
#     )
    
#     # Return the validation loss (minimized by Optuna)
#     return min(history.history['val_loss'])

# # Create and run the Optuna study
# study = optuna.create_study(direction="minimize")  # Minimize validation loss
# study.optimize(objective, n_trials=5)  # Adjust n_trials as needed

# # Display the best parameters
# print("Best hyperparameters:", study.best_params)

# # Build and train the final model with the best parameters
# best_params = study.best_params
# final_model = build_model(
#     trial=optuna.trial.FixedTrial(best_params),  # Use fixed parameters
#     input_shape=(X_train.shape[1], X_train.shape[2])
# )
# history = final_model.fit(
#     X_train, y_train,
#     validation_data=(X_val, y_val),
#     epochs=best_params['epochs'],  # Use optimal number of epochs
#     batch_size=16,
#     class_weight=class_weight_dict,
#     verbose=1
# )

# # Evaluate the model on the training, validation, and test sets
# train_loss, train_accuracy = final_model.evaluate(X_train, y_train, verbose=0)
# val_loss, val_accuracy = final_model.evaluate(X_val, y_val, verbose=0)
# test_loss, test_accuracy = final_model.evaluate(X_test, y_test, verbose=0)

# # Predict to calculate F1 scores
# y_train_pred = (final_model.predict(X_train) > 0.5).astype(int)
# y_val_pred = (final_model.predict(X_val) > 0.5).astype(int)
# y_test_pred = (final_model.predict(X_test) > 0.5).astype(int)

# train_f1 = f1_score(y_train, y_train_pred, average="weighted")
# val_f1 = f1_score(y_val, y_val_pred, average="weighted")
# test_f1 = f1_score(y_test, y_test_pred, average="weighted")

# # Print the final metrics
# print(f"Final Training Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Weighted F1 Score: {train_f1:.4f}")
# print(f"Final Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Weighted F1 Score: {val_f1:.4f}")
# print(f"Final Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}, Weighted F1 Score: {test_f1:.4f}")

# # Plot the training and validation loss over all epochs
# plt.figure(figsize=(10, 6))
# plt.plot(history.history['loss'], label="Training Loss")
# plt.plot(history.history['val_loss'], label="Validation Loss")
# plt.xlabel("Epochs")
# plt.ylabel("Loss")
# plt.title("Training and Validation Loss")
# plt.legend()
# plt.show()

# # Plot the training and validation F1 scores over all epochs
# # Add F1 scores to the history for tracking (retraining is required for this)
# # Assume a method to log F1 scores during training in a custom callback if needed.


In [ ]:
# Create and run the Optuna study
# study = optuna.create_study(direction="minimize")  # Minimize 1 - weighted F1 score
# study.optimize(objective, n_trials=5)  # Reduce n_trials for testing

# # Display the best parameters
# print("Best hyperparameters:", study.best_params)


In [ ]:
# # Train the final model with the best hyperparameters
# #best_params = study.best_params

## BEST PARAMS FOR F1 SCORE
# best_params = {'epochs': 130, 'embed_dim': 128, 'num_heads': 8, 'ff_dim': 256, 'num_blocks': 1, 'dropout_rate': 0.35062671547806423, 'l2_reg': 0.00010345073760147071, 'learning_rate': 0.00024969942836710864}


# # Build the final model with fixed parameters
# final_model = build_model(
#     trial=optuna.trial.FixedTrial(best_params),  # Use fixed trial with best params
#     input_shape=(X_train.shape[1], X_train.shape[2])
# )

# history = final_model.fit(
#     X_train, y_train,
#     validation_data=(X_val, y_val),
#     epochs=best_params['epochs'],  # Use optimal number of epochs
#     batch_size=16,
#     class_weight=class_weight_dict,
#     verbose=1
# )

# # Evaluate the model on the training, validation, and test sets
# train_loss, train_accuracy = final_model.evaluate(X_train, y_train, verbose=0)
# val_loss, val_accuracy = final_model.evaluate(X_val, y_val, verbose=0)
# test_loss, test_accuracy = final_model.evaluate(X_test, y_test, verbose=0)

# # Predict to calculate F1 scores
# y_train_pred = (final_model.predict(X_train) > 0.5).astype(int)
# y_val_pred = (final_model.predict(X_val) > 0.5).astype(int)
# y_test_pred = (final_model.predict(X_test) > 0.5).astype(int)

# train_f1 = f1_score(y_train, y_train_pred, average="weighted")
# val_f1 = f1_score(y_val, y_val_pred, average="weighted")
# test_f1 = f1_score(y_test, y_test_pred, average="weighted")

# # Print the final metrics
# print(f"Final Training Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.4f}, Weighted F1 Score: {train_f1:.4f}")
# print(f"Final Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Weighted F1 Score: {val_f1:.4f}")
# print(f"Final Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}, Weighted F1 Score: {test_f1:.4f}")

# # Plot the training and validation loss over all epochs
# plt.figure(figsize=(10, 6))
# plt.plot(history.history['loss'], label="Training Loss")
# plt.plot(history.history['val_loss'], label="Validation Loss")
# plt.xlabel("Epochs")
# plt.ylabel("Loss")
# plt.title("Training and Validation Loss")
# plt.legend()
# plt.show()



In [ ]:
# #### TRANSFORMER ARCHITECTURE!!!!! 

class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, regularizer=None):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.projection_dim = embed_dim // num_heads
        self.query_dense = Dense(embed_dim, kernel_regularizer=regularizer)
        self.key_dense = Dense(embed_dim, kernel_regularizer=regularizer)
        self.value_dense = Dense(embed_dim, kernel_regularizer=regularizer)
        self.combine_heads = Dense(embed_dim, kernel_regularizer=regularizer)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True) / tf.sqrt(tf.cast(self.projection_dim, query.dtype))
        weights = tf.nn.softmax(score, axis=-1)
        return tf.matmul(weights, value), weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        inputs = tf.cast(inputs, dtype=tf.float32)

        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)

        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        attention, _ = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        return self.combine_heads(concat_attention)

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, regularizer=None):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads, regularizer)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation='relu', kernel_regularizer=regularizer),
            Dense(embed_dim, kernel_regularizer=regularizer)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

def build_transformer_model(input_shape, num_heads=4, embed_dim=64, ff_dim=128, num_blocks=2, l2_reg=1e-3):
    inputs = Input(shape=input_shape)
    
    # Embedding layer with regularization
    x = Dense(embed_dim, kernel_regularizer=l2(l2_reg))(inputs)
    
    # Add Transformer blocks with regularization
    for _ in range(num_blocks):
        x = TransformerBlock(embed_dim, num_heads, ff_dim, rate=0.3, regularizer=l2(l2_reg))(x, training=True)

    # Global pooling and output layer
    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.5)(x) # INCREASED FROM 0.3
    outputs = Dense(1, activation="sigmoid", kernel_regularizer=l2(l2_reg))(x)

    model = Model(inputs=inputs, outputs=outputs)
    # learning rate decreaesd from 0.001
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

# Build the Transformer model
input_shape = (lstm_input.shape[1], lstm_input.shape[2])  # timesteps, features
transformer_model = build_transformer_model(input_shape)

# Summary of the model
transformer_model.summary()


In [ ]:
# Train the Transformer model
# from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=50,
    restore_best_weights=True
)
## MAYBE TRY BRINGING BACK EARLY STOPPING BUT WITH A DIFFERENT PATIENCE 
## MAYBE TRY CLASS WEIGHTS 

# Compute class weights to balance the dataset
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# lr_scheduler = ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.5,  # Reduce the learning rate by half
#     patience=5,  # Wait for 5 epochs without improvement
#     min_lr=1e-6  # Minimum learning rate
# )

history = transformer_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=70,
    batch_size=16,
    callbacks=[early_stopping], # Trying bringing back early stopping or lr_scheduler
    class_weight=class_weight_dict,
    verbose=1
)

train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Print final training and validation accuracy
print(f"Final Training Accuracy: {train_accuracy[-1]:.4f}")
print(f"Final Validation Accuracy: {val_accuracy[-1]:.4f}")

# Evaluate the model on test data
test_loss, test_accuracy = transformer_model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Get predictions
y_pred = (transformer_model.predict(X_test) > 0.5).astype("int32")  # Threshold at 0.5

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))

test_f1 = f1_score(y_test, y_pred, average="weighted")
print("Weighted F1 Score: ", test_f1)

In [ ]:
import matplotlib.pyplot as plt

# Plot Training Loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs - Kinematic Timeseries')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Print final training and validation accuracy
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
print(f"Final Training Accuracy: {train_accuracy[-1]:.4f}")
print(f"Final Validation Accuracy: {val_accuracy[-1]:.4f}")



# Evaluate the model on test data
test_loss, test_accuracy = transformer_model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Get predictions
y_pred_prob = transformer_model.predict(X_test).ravel()  # Predicted probabilities
y_pred = (y_pred_prob > 0.5).astype("int32")  # Threshold at 0.5

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=4))

weighted_recall = classification_report(y_test, (y_pred_prob > 0.5).astype(int), output_dict=True)['weighted avg']['recall']
weighted_precision = classification_report(y_test, (y_pred_prob > 0.5).astype(int), output_dict=True)['weighted avg']['precision']

print('weighted average precision: ', weighted_precision)
print("average recall: ", weighted_recall)


# Compute and print Weighted F1 Score
test_f1 = f1_score(y_test, y_pred, average="weighted")
print("Weighted F1 Score: ", test_f1)

# ROC-AUC and Precision-Recall Curves
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
average_precision = average_precision_score(y_test, y_pred_prob)

# Plot ROC Curve
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})', color='blue')
plt.plot([0, 1], [0, 1], 'k--', label='Chance')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')

# Plot Precision-Recall Curve
plt.subplot(1, 2, 2)
plt.plot(recall, precision, label=f'PR curve (AUPRC = {average_precision:.2f})', color='green')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall (PR) Curve')
plt.legend(loc='lower left')

plt.tight_layout()
plt.show()

# Print ROC-AUC and PR-AUC scores
print(f"\nFinal ROC-AUC: {roc_auc:.4f}")
print(f"Final Precision-Recall AUC (AUPRC): {average_precision:.4f}")

In [ ]:
from tensorflow.keras.utils import plot_model

# Assuming `transformer_model` is the built model from your code
# Save the architecture diagram to a file
plot_model(transformer_model, to_file='transformer_model_architecture.png', 
           show_shapes=True, show_layer_names=True, expand_nested=True, dpi=96)

# Display the diagram inline (requires the file saved above)
from PIL import Image
import matplotlib.pyplot as plt

# Load and display the saved image
img = Image.open('transformer_model_architecture.png')
plt.figure(figsize=(10, 15))
plt.imshow(img)
plt.axis('off')  # Hide axes for better visualization
plt.show()


In [ ]:
# ## Trying to Find the Ideal Hyperparameters Using Grid Search


# # Define the model-building function
# def build_model(embed_dim, num_heads, ff_dim, num_blocks, l2_reg, dropout_rate):
#     inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))
#     x = Dense(embed_dim, kernel_regularizer=l2(l2_reg))(inputs)
#     for _ in range(num_blocks):
#         x = TransformerBlock(embed_dim, num_heads, ff_dim, rate=dropout_rate, regularizer=l2(l2_reg))(x, training=True)
#     x = GlobalAveragePooling1D()(x)
#     x = Dropout(dropout_rate)(x)
#     outputs = Dense(1, activation="sigmoid", kernel_regularizer=l2(l2_reg))(x)

#     model = Model(inputs=inputs, outputs=outputs)
#     model.compile(optimizer=Adam(learning_rate=0.001),
#                   loss='binary_crossentropy',
#                   metrics=['accuracy'])
#     return model

# # Wrap the Keras model for GridSearchCV
# model = KerasClassifier(
#     model=build_model,
#     embed_dim=64,  # Default value, will be overridden by GridSearch
#     num_heads=4,
#     ff_dim=128,
#     num_blocks=2,
#     l2_reg=1e-3,
#     dropout_rate=0.3,
#     epochs=50,  # Use fewer epochs for tuning
#     batch_size=16,
#     verbose=0
# )

# # Define the parameter grid
# param_grid = {
#     'embed_dim': [32, 64, 128],
#     'num_heads': [2, 4, 6],
#     'ff_dim': [64, 128, 256],
#     'num_blocks': [1, 2, 3],
#     'l2_reg': [1e-3, 1e-4],
#     'dropout_rate': [0.1, 0.3, 0.5]
# }

# # Perform the grid search
# grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)  # 3-fold cross-validation
# grid_result = grid.fit(X_train, y_train)

# # Print the best parameters and score
# print("Best parameters:", grid_result.best_params_)
# print("Best validation accuracy:", grid_result.best_score_)


In [ ]:
# import optuna
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Dense, Dropout, Input, GlobalAveragePooling1D
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.regularizers import l2
# import tensorflow as tf

# # Define the Transformer-based model-building function
# def build_model(trial, input_shape):
#     # Suggest hyperparameters
#     embed_dim = trial.suggest_int("embed_dim", 32, 128, step=32)  # Embedding dimension
#     num_heads = trial.suggest_int("num_heads", 2, 8, step=2)      # Number of attention heads
#     ff_dim = trial.suggest_int("ff_dim", 64, 256, step=64)        # Feedforward network dimension
#     num_blocks = trial.suggest_int("num_blocks", 1, 3)            # Number of transformer blocks
#     dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)  # Dropout rate
#     l2_reg = trial.suggest_loguniform("l2_reg", 1e-4, 1e-2)       # L2 regularization strength

#     # Input layer
#     inputs = Input(shape=input_shape)
    
#     # Dense embedding layer
#     x = Dense(embed_dim, kernel_regularizer=l2(l2_reg))(inputs)
    
#     # Add Transformer blocks
#     for _ in range(num_blocks):
#         x = TransformerBlock(embed_dim, num_heads, ff_dim, rate=dropout_rate, regularizer=l2(l2_reg))(x, training=True)
    
#     # Global pooling and output layer
#     x = GlobalAveragePooling1D()(x)
#     x = Dropout(dropout_rate)(x)
#     outputs = Dense(1, activation="sigmoid", kernel_regularizer=l2(l2_reg))(x)
    
#     # Build and compile the model
#     model = Model(inputs=inputs, outputs=outputs)
#     model.compile(optimizer=Adam(learning_rate=trial.suggest_loguniform("learning_rate", 1e-4, 1e-2)),
#                   loss='binary_crossentropy',
#                   metrics=['accuracy'])
#     return model



In [ ]:
# # Get the best hyperparameters
# #best_params = study.best_params

# best_params = {'epochs': 150, 'embed_dim': 128, 'num_heads': 8, 'ff_dim': 256, '
# num_blocks': 1, 'dropout_rate': 0.35062671547806423, 'l2_reg': 0.00010345073760147071, 'learning_rate': 0.00024969942836710864}

# # Build the final model using the best hyperparameters
# final_model = build_model(
#     trial=optuna.trial.FixedTrial(best_params),  # Use fixed trial with best params
#     input_shape=(X_train.shape[1], X_train.shape[2])
# )

# # Train the final model
# final_model.fit(
#     X_train, y_train,
#     validation_data=(X_val, y_val),
#     epochs=500,  # Use full training time
#     batch_size=16,
#     verbose=1
# )

# # Evaluate on the test set
# test_loss, test_accuracy = final_model.evaluate(X_test, y_test, verbose=1)
# print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


In [ ]:
### LSTM attempts - did not work

In [ ]:
# # Define function to normalize joint angles across all data
# def normalize_data(data_list):
#     """
#     Normalizes a list of numpy arrays using MinMax scaling across all features.
#     Arguments:
#         data_list: List of numpy arrays where each array corresponds to one time series.
#     Returns:
#         normalized_data: List of normalized numpy arrays.
#     """
#     # Flatten all data for joint normalization across the dataset
#     all_data = np.array(data_list)
    
#     # Apply MinMaxScaler
#     scaler = MinMaxScaler()
#     scaler.fit(all_data)  # Fit scaler on all joint angles across the dataset
    
#     # Normalize each time series
#     normalized_data = [scaler.transform(data) for data in data_list]
#     return normalized_data, scaler

# # Define function to pad time series to the same length
# def pad_time_series(data_list, max_length=None):
#     """
#     Pads all time series in data_list to the same length.
#     Arguments:
#         data_list: List of numpy arrays where each array corresponds to one time series.
#         max_length: If None, pad to the longest time series. Otherwise, pad to max_length.
#     Returns:
#         padded_data: Numpy array of shape (num_samples, max_length, num_features).
#     """
#     if max_length is None:
#         max_length = max(len(data) for data in data_list)  # Find the longest time series
#     padded_data = pad_sequences(data_list, maxlen=max_length, padding="post", dtype="float32")
#     return padded_data

In [ ]:
# def build_lstm_model_2(input_shape):
#     model = Sequential([
#         Masking(mask_value=0.0, input_shape=input_shape),  # Masking for padded values
#         LSTM(128, return_sequences=True),  # First LSTM layer with increased units
#         BatchNormalization(),  # Batch normalization to stabilize training
#         Dropout(0.2),  # Reduced dropout rate
#         LSTM(64, return_sequences=False),  # Second LSTM layer
#         BatchNormalization(),  # Batch normalization after second LSTM layer
#         Dropout(0.2),
#         Dense(32, activation='relu'),  # Increased fully connected layer size
#         Dense(1, activation='sigmoid')  # Output layer for binary classification
#     ])
#     model.compile(
#         optimizer=Adam(learning_rate=0.0001),  # Reduced learning rate
#         loss='binary_crossentropy',
#         metrics=['accuracy']
#     )
#     return model

In [ ]:


# # Ensure consistent shuffling for reproducibility
# np.random.seed(42)

# def manual_train_val_test_split(data, labels, train_ratio=0.8, val_ratio=0.1):
#     """
#     Manually splits data and labels into training, validation, and testing sets with the given ratios.
#     Arguments:
#         data: Preprocessed time series data (numpy array).
#         labels: Corresponding labels (numpy array).
#         train_ratio: Proportion of data to use for training (default 0.7).
#         val_ratio: Proportion of data to use for validation (default 0.2).
#     Returns:
#         X_train, X_val, X_test, y_train, y_val, y_test: Split data and labels.
#     """
#     # Ensure train_ratio + val_ratio < 1
#     assert train_ratio + val_ratio < 1, "Train and validation ratios must sum to less than 1."
    
#     # Shuffle the indices
#     indices = np.arange(len(data))
#     np.random.shuffle(indices)
    
#     # Calculate the split points
#     train_split = int(len(data) * train_ratio)
#     val_split = int(len(data) * (train_ratio + val_ratio))
    
#     # Split the data and labels
#     train_indices = indices[:train_split]
#     val_indices = indices[train_split:val_split]
#     test_indices = indices[val_split:]
    
#     X_train = data[train_indices]
#     X_val = data[val_indices]
#     X_test = data[test_indices]
#     y_train = labels[train_indices]
#     y_val = labels[val_indices]
#     y_test = labels[test_indices]
    
#     return X_train, X_val, X_test, y_train, y_val, y_test


In [ ]:
# Define LSTM model
# def build_lstm_model_1(input_shape):
#     model = Sequential([
#         Masking(mask_value=0.0, input_shape=input_shape),  # Masking for padded values
#         LSTM(64, return_sequences=True),  # First LSTM layer
#         Dropout(0.3),  # Dropout for regularization
#         LSTM(32),  # Second LSTM layer
#         Dropout(0.3),
#         Dense(16, activation='relu'),  # Fully connected layer
#         Dense(1, activation='sigmoid')  # Output layer for binary classification
#     ])
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
#         loss='binary_crossentropy',
#         metrics=['accuracy']
#     )
#     return model


In [ ]:
# ### Perform the manual train-test split
# # Call the function
# X_train, X_val, X_test, y_train, y_val, y_test = manual_train_val_test_split(
#     np.array(lstm_input), np.array(labels), train_ratio=0.8, val_ratio=0.1)
# # Check the shapes of the resulting datasets
# print("Training data shape:", X_train.shape)
# print("Training labels shape:", y_train.shape)
# print("Testing data shape:", X_test.shape)
# print("Testing labels shape:", y_test.shape)


# # Convert to numpy arrays
# X_train = np.array(X_train)
# X_val = np.array(X_val)
# X_test = np.array(X_test)
# y_train = np.array(y_train)
# y_val = np.array(y_val)
# y_test = np.array(y_test)

# # Define input shape (timepoints, features)
# input_shape = (X_train.shape[1], X_train.shape[2])

# # # Build model
# # model = build_lstm_model(input_shape)

# # # Train the model
# # history = model.fit(
# #     X_train, y_train,
# #     validation_data=(X_val, y_val),
# #     epochs=50,
# #     batch_size=16,
# #     verbose=1
# # )




In [ ]:
# TRY 3 
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Masking, LSTM, Dropout, Dense, BatchNormalization
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping
# from sklearn.utils.class_weight import compute_class_weight
# import numpy as np

# # Define LSTM model with improvements
# def build_lstm_model(input_shape):
#     model = Sequential([
#         Masking(mask_value=0.0, input_shape=input_shape),  # Masking for padded values
#         LSTM(128, return_sequences=True),  # Increased units in the first LSTM layer
#         BatchNormalization(),  # Batch Normalization
#         Dropout(0.4),  # Increased Dropout for regularization
#         LSTM(64),  # Second LSTM layer with reduced units
#         BatchNormalization(),  # Batch Normalization
#         Dropout(0.4),
#         Dense(32, activation='relu'),  # Increased units in fully connected layer
#         Dropout(0.4),  # Additional dropout
#         Dense(1, activation='sigmoid')  # Output layer for binary classification
#     ])
#     model.compile(
#         optimizer=Adam(learning_rate=0.0001),  # Reduced learning rate
#         loss='binary_crossentropy',
#         metrics=['accuracy']
#     )
#     return model

# # Build the model
# input_shape = (X_train.shape[1], X_train.shape[2])  # Define input shape based on your data
# model = build_lstm_model(input_shape)

# # Compute class weights for handling class imbalance
# class_weights = compute_class_weight(
#     'balanced',
#     classes=np.unique(y_train),
#     y=y_train
# )
# class_weights = dict(enumerate(class_weights))

# # Add EarlyStopping for regularization
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# # Train the model with validation and early stopping
# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_val, y_val),
#     epochs=50,
#     batch_size=16,
#     class_weight=class_weights,
#     callbacks=[early_stopping],
#     verbose=1
# )

# # Evaluate the model on test data
# test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
# print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")


In [ ]:
# # THE LATEST LSTM

# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Masking, LSTM, Dense, Dropout, BatchNormalization, Input
#from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report

# # Define the LSTM model
# def build_lstm_model(input_shape):
#     model = Sequential([
#         Input(shape=input_shape),
#         LSTM(256, return_sequences=True),  # Increase units
#         Dense(128, activation='relu'),  # Add a dense layer
#         LSTM(128, return_sequences=True),  # Second LSTM layer
#         Dense(64, activation='relu'),  # Add another dense layer
#         LSTM(64),  # Final LSTM layer
#         Dense(32, activation='relu'),  # Fully connected layer
#         Dense(1, activation='sigmoid')  # Output layer for binary classification
#     ])
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),  # Higher LR
#         loss='binary_crossentropy',
#         metrics=['accuracy']
#     )
#     return model


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Masking, LSTM, Dense, Dropout, BatchNormalization
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report

# # Build an LSTM model optimized for overfitting to the training set
# def build_overfitting_lstm(input_shape):
#     model = Sequential([
#         Masking(mask_value=0.0, input_shape=input_shape),  # Masking padded values
#         LSTM(128, return_sequences=True, activation='relu'),  # First LSTM layer
#         LSTM(64, return_sequences=True, activation='relu'),   # Second LSTM layer
#         LSTM(32, activation='relu'),                         # Third LSTM layer
#         Dense(1, activation='sigmoid')                       # Output layer for binary classification
#     ])
#     model.compile(
#         optimizer=Adam(learning_rate=0.001),
#         loss='binary_crossentropy',
#         metrics=['accuracy']
#     )
#     return model

# # Train the model on training data only
# input_shape = (X_train.shape[1], X_train.shape[2])  # Define input shape from the training data
# model = build_overfitting_lstm(input_shape)

# # Train the model for an extended number of epochs
# history = model.fit(
#     X_train, y_train,
#     epochs=100,  # Increase epochs for training
#     batch_size=16,
#     verbose=1
# )

# # Plot training loss over epochs
# plt.plot(history.history['loss'], label='Train Loss')
# plt.title('Training Loss Over Epochs')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()


In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Input
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import LearningRateScheduler

# def build_lstm_model(input_shape):
#     model = Sequential([
#         Input(shape=input_shape),
#         LSTM(256, return_sequences=True),  # Larger capacity
#         LSTM(128, return_sequences=True),
#         LSTM(128, return_sequences=True),
#         LSTM(64),
#         Dense(32, activation='relu'),
#         Dense(1, activation='sigmoid')
#     ])
#     model.compile(
#         optimizer=Adam(learning_rate=0.0001),
#         loss='binary_crossentropy',
#         metrics=['accuracy']
#     )
#     return model


# # Build model
# input_shape = (X_train.shape[1], X_train.shape[2])  # Define input shape from the training data
# model = build_lstm_model(input_shape)


# # Train the model with class weights
# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_test, y_test),
#     epochs=500,
#     batch_size=16,
#     verbose=1
# )




In [ ]:
# import matplotlib.pyplot as plt

# # Extract the loss values from the history object
# train_loss = history.history['loss']
# val_loss = history.history['val_loss']

# # Print loss values for each epoch
# print("Training Loss per Epoch:")
# for epoch, loss in enumerate(train_loss):
#     print(f"Epoch {epoch+1}: Training Loss = {loss:.4f}, Validation Loss = {val_loss[epoch]:.4f}")

# # Plot the training and validation loss
# plt.figure(figsize=(8, 6))
# plt.plot(train_loss, label='Training Loss', marker='o')
# plt.plot(val_loss, label='Validation Loss', marker='o')
# plt.title('Training and Validation Loss Over Epochs')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()
# plt.grid()
# plt.show()
